In [7]:
# import numpy as np
# import torch
# import torch.nn as nn
# import torchvision
# from torch.utils.data import DataLoader
# from torchvision import datasets
# import torchvision.transforms as transforms
# import os
# import time
# import sys
# import torch.quantization

# # # warnings 설정
# import warnings
# warnings.filterwarnings(
#     action='ignore',
#     category=DeprecationWarning,
#     module=r'.*'
# )
# warnings.filterwarnings(
#     action='default',
#     module=r'torch.quantization'
# )

# # 반복 가능한 결과를 위한 랜덤 시드 지정하기
# torch.manual_seed(191009)

# from torch.quantization import QuantStub, DeQuantStub

### 1. Model Architecture

In [ ]:
# from torch.quantization import QuantStub, DeQuantStub

# def _make_divisible(v, divisor, min_value=None):
#     """
#     이 함수는 원본 TensorFlow 저장소에서 가져왔습니다.
#     모든 계층이 8로 나누어지는 채널 숫자를 가지고 있습니다.
#     이곳에서 확인 가능합니다:
#     https://github.com/tensorflow/models/blob/master/research/slim/nets/mobilenet/mobilenet.py
#     :param v:
#     :param divisor:
#     :param min_value:
#     :return:
#     """
#     if min_value is None:
#         min_value = divisor
#     new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
#     # 내림은 10% 넘게 내려가지 않는 것을 보장합니다.
#     if new_v < 0.9 * v:
#         new_v += divisor
#     return new_v


# class ConvBNReLU(nn.Sequential):
#     def __init__(self, in_planes, out_planes, kernel_size=3, stride=1, groups=1):
#         padding = (kernel_size - 1) // 2
#         super(ConvBNReLU, self).__init__(
#             nn.Conv2d(in_planes, out_planes, kernel_size, stride, padding, groups=groups, bias=False),
#             nn.BatchNorm2d(out_planes, momentum=0.1),
#             # ReLU로 교체
#             nn.ReLU(inplace=False)
#         )


# class InvertedResidual(nn.Module):
#     def __init__(self, inp, oup, stride, expand_ratio):
#         super(InvertedResidual, self).__init__()
#         self.stride = stride
#         assert stride in [1, 2]

#         hidden_dim = int(round(inp * expand_ratio))
#         self.use_res_connect = self.stride == 1 and inp == oup

#         layers = []
#         if expand_ratio != 1:
#             # pw
#             layers.append(ConvBNReLU(inp, hidden_dim, kernel_size=1))
#         layers.extend([
#             # dw
#             ConvBNReLU(hidden_dim, hidden_dim, stride=stride, groups=hidden_dim),
#             # pw-linear
#             nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
#             nn.BatchNorm2d(oup, momentum=0.1),
#         ])
#         self.conv = nn.Sequential(*layers)
#         # torch.add를 floatfunctional로 교체
#         self.skip_add = nn.quantized.FloatFunctional()

#     def forward(self, x):
#         if self.use_res_connect:
#             return self.skip_add.add(x, self.conv(x))
#         else:
#             return self.conv(x)


# class MobileNetV2(nn.Module):
#     def __init__(self, num_classes=1000, width_mult=1.0, inverted_residual_setting=None, round_nearest=8):
#         """
#         MobileNet V2 메인 클래스
#         Args:
#             num_classes (int): 클래스 숫자
#             width_mult (float): 넓이 multiplier - 이 수를 통해 각 계층의 채널 개수를 조절
#             inverted_residual_setting: 네트워크 구조
#             round_nearest (int): 각 계층의 채널 숫를 이 숫자의 배수로 반올림
#             1로 설정하면 반올림 정지
#         """
#         super(MobileNetV2, self).__init__()
#         block = InvertedResidual
#         input_channel = 32
#         last_channel = 1280

#         if inverted_residual_setting is None:
#             inverted_residual_setting = [
#                 # t, c, n, s
#                 [1, 16, 1, 1],
#                 [6, 24, 2, 2],
#                 [6, 32, 3, 2],
#                 [6, 64, 4, 2],
#                 [6, 96, 3, 1],
#                 [6, 160, 3, 2],
#                 [6, 320, 1, 1],
#             ]

#         # 사용자가 t,c,n,s를 필요하다는 것을 안다는 전제하에 첫 번째 요소만 확인
#         if len(inverted_residual_setting) == 0 or len(inverted_residual_setting[0]) != 4:
#             raise ValueError("inverted_residual_setting should be non-empty "
#                              "or a 4-element list, got {}".format(inverted_residual_setting))

#         # 첫 번째 계층 만들기
#         input_channel = _make_divisible(input_channel * width_mult, round_nearest)
#         self.last_channel = _make_divisible(last_channel * max(1.0, width_mult), round_nearest)
#         features = [ConvBNReLU(3, input_channel, stride=2)]
#         # 역전된 잔차 블럭(inverted residual blocks) 만들기
#         for t, c, n, s in inverted_residual_setting:
#             output_channel = _make_divisible(c * width_mult, round_nearest)
#             for i in range(n):
#                 stride = s if i == 0 else 1
#                 features.append(block(input_channel, output_channel, stride, expand_ratio=t))
#                 input_channel = output_channel
#         # 마지막 계층들 만들기
#         features.append(ConvBNReLU(input_channel, self.last_channel, kernel_size=1))
#         # nn.Sequential로 만들기
#         self.features = nn.Sequential(*features)
#         self.quant = QuantStub()
#         self.dequant = DeQuantStub()
#         # 분류기(classifier) 만들기
#         self.classifier = nn.Sequential(
#             nn.Dropout(0.2),
#             nn.Linear(self.last_channel, num_classes),
#         )

#         # 가중치 초기화
#         for m in self.modules():
#             if isinstance(m, nn.Conv2d):
#                 nn.init.kaiming_normal_(m.weight, mode='fan_out')
#                 if m.bias is not None:
#                     nn.init.zeros_(m.bias)
#             elif isinstance(m, nn.BatchNorm2d):
#                 nn.init.ones_(m.weight)
#                 nn.init.zeros_(m.bias)
#             elif isinstance(m, nn.Linear):
#                 nn.init.normal_(m.weight, 0, 0.01)
#                 nn.init.zeros_(m.bias)

#     def forward(self, x):

#         x = self.quant(x)

#         x = self.features(x)
#         x = x.mean([2, 3])
#         x = self.classifier(x)
#         x = self.dequant(x)
#         return x

#     # 양자화 전에 Conv+BN과 Conv+BN+Relu 모듈 결합(fusion)
#     # 이 연산은 숫자를 변경하지 않음
#     def fuse_model(self):
#         for m in self.modules():
#             if type(m) == ConvBNReLU:
#                 torch.quantization.fuse_modules(m, ['0', '1', '2'], inplace=True)
#             if type(m) == InvertedResidual:
#                 for idx in range(len(m.conv)):
#                     if type(m.conv[idx]) == nn.Conv2d:
#                         torch.quantization.fuse_modules(m.conv, [str(idx), str(idx + 1)], inplace=True)

### 2. Helper Functions

In [ ]:
# class AverageMeter(object):
#     """평균과 현재 값 계산 및 저장"""
#     def __init__(self, name, fmt=':f'):
#         self.name = name
#         self.fmt = fmt
#         self.reset()

#     def reset(self):
#         self.val = 0
#         self.avg = 0
#         self.sum = 0
#         self.count = 0

#     def update(self, val, n=1):
#         self.val = val
#         self.sum += val * n
#         self.count += n
#         self.avg = self.sum / self.count

#     def __str__(self):
#         fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
#         return fmtstr.format(**self.__dict__)


# def accuracy(output, target, topk=(1,)):
#     """특정 k값을 위해 top k 예측의 정확도 계산"""
#     with torch.no_grad():
#         maxk = max(topk)
#         batch_size = target.size(0)

#         _, pred = output.topk(maxk, 1, True, True)
#         pred = pred.t()
#         correct = pred.eq(target.view(1, -1).expand_as(pred))

#         res = []
#         for k in topk:
#             correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
#             res.append(correct_k.mul_(100.0 / batch_size))
#         return res


# def evaluate(model, criterion, data_loader, neval_batches):
#     model.eval()
#     top1 = AverageMeter('Acc@1', ':6.2f')
#     top5 = AverageMeter('Acc@5', ':6.2f')
#     cnt = 0
#     with torch.no_grad():
#         for image, target in data_loader:
#             output = model(image)
#             loss = criterion(output, target)
#             cnt += 1
#             acc1, acc5 = accuracy(output, target, topk=(1, 5))
#             print('.', end = '')
#             top1.update(acc1[0], image.size(0))
#             top5.update(acc5[0], image.size(0))
#             if cnt >= neval_batches:
#                  return top1, top5

#     return top1, top5

# def load_model(model_file):
#     model = MobileNetV2()
#     state_dict = torch.load(model_file)
#     model.load_state_dict(state_dict)
#     model.to('cpu')
#     return model

# def print_size_of_model(model):
#     torch.save(model.state_dict(), "temp.p")
#     print('Size (MB):', os.path.getsize("temp.p")/1e6)
#     os.remove('temp.p')

### 3. Define Dataset and DataLoader

In [ ]:
# def prepare_data_loaders(data_path):

#     normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                      std=[0.229, 0.224, 0.225])
#     dataset = torchvision.datasets.ImageNet(
#            data_path, split="train",
#          transforms.Compose([
#                    transforms.RandomResizedCrop(224),
#                    transforms.RandomHorizontalFlip(),
#                    transforms.ToTensor(),
#                    normalize,
#                ]))
#     dataset_test = torchvision.datasets.ImageNet(
#           data_path, split="val",
#               transforms.Compose([
#                   transforms.Resize(256),
#                   transforms.CenterCrop(224),
#                   transforms.ToTensor(),
#                   normalize,
#               ]))

#     train_sampler = torch.utils.data.RandomSampler(dataset)
#     test_sampler = torch.utils.data.SequentialSampler(dataset_test)

#     data_loader = torch.utils.data.DataLoader(
#         dataset, batch_size=train_batch_size,
#         sampler=train_sampler)

#     data_loader_test = torch.utils.data.DataLoader(
#         dataset_test, batch_size=eval_batch_size,
#         sampler=test_sampler)

#     return data_loader, data_loader_test

---

In [17]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as transforms
import os
import time
import sys
import torch.quantization

# # warnings 설정
import warnings
warnings.filterwarnings(
    action='ignore',
    category=DeprecationWarning,
    module=r'.*'
)
warnings.filterwarnings(
    action='default',
    module=r'torch.quantization'
)

# 반복 가능한 결과를 위한 랜덤 시드 지정하기
torch.manual_seed(191009)

from torch.quantization import QuantStub, DeQuantStub

from model import FasterRCNNVGG16
from model import faster_rcnn
from trainer import FasterRCNNTrainer

In [20]:
class AverageMeter(object):
    """평균과 현재 값 계산 및 저장"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


def accuracy(output, target, topk=(1,)):
    """특정 k값을 위해 top k 예측의 정확도 계산"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res


def evaluate(model, criterion, data_loader, neval_batches):
    model.eval()
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    cnt = 0
    with torch.no_grad():
        for image, target in data_loader:
            output = model(image)
            loss = criterion(output, target)
            cnt += 1
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            print('.', end = '')
            top1.update(acc1[0], image.size(0))
            top5.update(acc5[0], image.size(0))
            if cnt >= neval_batches:
                 return top1, top5

    return top1, top5

def load_model(model_file):
    # model = MobileNetV2()
    model = FasterRCNNVGG16()
    model.load_state_dict(torch.load(model_file), strict=False)
    model.to('cpu')
    return model

def print_size_of_model(model):
    torch.save(model.state_dict(), "temp.p")
    print('Size (MB):', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')

In [21]:
def train_one_epoch(model, criterion, optimizer, data_loader, device, ntrain_batches):
    model.train()
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    avgloss = AverageMeter('Loss', '1.5f')

    cnt = 0
    for image, target in data_loader:
        start_time = time.time()
        print('.', end = '')
        cnt += 1
        image, target = image.to(device), target.to(device)
        output = model(image)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        top1.update(acc1[0], image.size(0))
        top5.update(acc5[0], image.size(0))
        avgloss.update(loss, image.size(0))
        if cnt >= ntrain_batches:
            print('Loss', avgloss.avg)

            print('Training: * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
                  .format(top1=top1, top5=top5))
            return

    print('Full imagenet train set:  * Acc@1 {top1.global_avg:.3f} Acc@5 {top5.global_avg:.3f}'
          .format(top1=top1, top5=top5))
    return

In [34]:
# data_path = '~/.data/imagenet'
saved_model_dir = 'checkpoints/'
float_model_file = 'tmp.pt'
scripted_float_model_file = 'fasterrcnn_quantization_scripted.pt'
scripted_quantized_model_file = 'fasterrcnn_quantization_scripted_quantized.pt'

# train_batch_size = 30
# eval_batch_size = 50
train_batch_size = 1
eval_batch_size = 1

# data_loader, data_loader_test = prepare_data_loaders(data_path)
criterion = nn.CrossEntropyLoss()
float_model = load_model(saved_model_dir + float_model_file).to('cpu')

# 다음으로 "모듈 결합"을 합니다. 모듈 결합은 메모리 접근을 줄여 모델을 빠르게 만들면서
# 정확도 수치를 향상시킵니다. 모듈 결합은 어떠한 모델에라도 사용할 수 있지만,
# 양자화된 모델에 사용하는 것이 특히나 더 일반적입니다.
# print('\n Inverted Residual Block: Before fusion \n\n', float_model.features[1].conv)
print('\n Inverted Residual Block: Before fusion \n\n', float_model.extractor[0])
float_model.eval()

# 모듈 결합
float_model.fuse_model()

# Conv+BN+Relu와 Conv+Relu 결합에 유의
# print('\n Inverted Residual Block: After fusion\n\n',float_model.features[1].conv)
print('\n Inverted Residual Block: After fusion\n\n',float_model.extractor[0])


 Inverted Residual Block: Before fusion 

 Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

 Inverted Residual Block: After fusion

 Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [38]:
qat_model = load_model(saved_model_dir + float_model_file)

modules_to_fuse = [['extractor', 'rpn', 'head']]
qat_model = torch.quantization.fuse_modules(qat_model, ['Sequential'])
# qat_model.fuse_model()

# optimizer = torch.optim.SGD(qat_model.parameters(), lr = 0.0001)
# qat_model.qconfig = torch.quantization.get_default_qat_qconfig('fbgemm')

AttributeError: 'FasterRCNNVGG16' object has no attribute 'Sequential'

***

### New New New Approach

In [58]:
# import torch

# # define a floating point model where some layers could benefit from QAT
# class M(torch.nn.Module):
#     def __init__(self):
#         super(M, self).__init__()
#         # QuantStub converts tensors from floating point to quantized
#         self.quant = torch.quantization.QuantStub()
#         self.conv = torch.nn.Conv2d(1, 1, 1)
#         self.bn = torch.nn.BatchNorm2d(1)
#         self.relu = torch.nn.ReLU()
#         # DeQuantStub converts tensors from quantized to floating point
#         self.dequant = torch.quantization.DeQuantStub()

#     def forward(self, x):
#         x = self.quant(x)
#         x = self.conv(x)
#         x = self.bn(x)
#         x = self.relu(x)
#         x = self.dequant(x)
#         return x

# # create a model instance
# model_fp32 = M()

# # model must be set to train mode for QAT logic to work
# model_fp32.train()

# # attach a global qconfig, which contains information about what kind
# # of observers to attach. Use 'fbgemm' for server inference and
# # 'qnnpack' for mobile inference. Other quantization configurations such
# # as selecting symmetric or assymetric quantization and MinMax or L2Norm
# # calibration techniques can be specified here.
# model_fp32.qconfig = torch.quantization.get_default_qat_qconfig('fbgemm')

# # fuse the activations to preceding layers, where applicable
# # this needs to be done manually depending on the model architecture
# model_fp32_fused = torch.quantization.fuse_modules(model_fp32,
#     [['conv', 'bn', 'relu']])

# # Prepare the model for QAT. This inserts observers and fake_quants in
# # the model that will observe weight and activation tensors during calibration.
# model_fp32_prepared = torch.quantization.prepare_qat(model_fp32_fused)

# # run the training loop (not shown)
# training_loop(model_fp32_prepared)

# # Convert the observed model to a quantized model. This does several things:
# # quantizes the weights, computes and stores the scale and bias value to be
# # used with each activation tensor, fuses modules where appropriate,
# # and replaces key operators with quantized implementations.
# model_fp32_prepared.eval()
# model_int8 = torch.quantization.convert(model_fp32_prepared)

# # run the model, relevant calculations will happen in int8
# res = model_int8(input_fp32)

/opt/conda/lib/python3.8/site-packages/torch/ao/quantization/observer.py:177: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


NameError: name 'training_loop' is not defined